In [781]:
import pandas as pd
import datetime
import re

In [782]:
df_baniola=pd.read_csv('baniola1.csv')
df_automobile=pd.read_csv('automobile_data.csv')

In [783]:
print(df_baniola.head())
print('--------------------------------------------------------------------')
print(df_automobile.head())

    Marque   Modèle  year horsepower        road     boite     fuel cylindrée  \
0     Fiat  FIORINO  2020       5 CV      180 KM  Manuelle   Diesel       4 L   
1  Peugeot      308  2015       5 CV  180 000 KM  Manuelle  Essence       3 L   
2  Renault   Kangoo  2012       5 CV  240 000 KM  Manuelle   Diesel       6 L   
3   Suzuki   Autres  2020       5 CV   90 000 KM  Manuelle  Essence       3 L   
4   Suzuki   Autres  2020       5 CV   88 000 KM  Manuelle  Essence       3 L   

   nbre_portes  Negociable   Prix  
0          3.0  Négociable     30  
1          5.0  Négociable  32000  
2          3.0         NaN  24500  
3          5.0  Négociable  35800  
4          5.0  Négociable  36500  
--------------------------------------------------------------------
          Marque             Modèle  year        road        boite horsepower  \
0  Mercedes-Benz           Classe E  2024        0 km  Automatique      10 cv   
1        Peugeot               2008  2014  162 000 km     Manuelle

In [784]:
# ajuster le feature 'cylindée' de df_automobile pour qu'il soit de type int:
df_automobile['cylindrée'] = df_automobile['cylindrée'].astype(float)/1000
print(df_automobile.head())
df_baniola['cylindrée'] = df_baniola['cylindrée'].apply(lambda x: re.search(r'\d+', x).group() if pd.notnull(x) else x)
print(df_baniola.head())

          Marque             Modèle  year        road        boite horsepower  \
0  Mercedes-Benz           Classe E  2024        0 km  Automatique      10 cv   
1        Peugeot               2008  2014  162 000 km     Manuelle       5 cv   
2          Mazda                  6  2013  256 000 km  Automatique       9 cv   
3           Ford  Ecosport Titanium  2022   60 000 km     Manuelle       7 cv   
4         Toyota         C-HR Sport  2020  162 000 km  Automatique       6 cv   

      fuel  nbre_portes  cylindrée transmission     Prix  
0  Essence          4.0      1.400    Intégrale  430 000  
1  Essence          4.0      2.398     Traction   36 000  
2  Essence          5.0      0.999     Traction   33 000  
3  Essence          5.0      1.497     Traction   68 000  
4  Essence          4.0      1.999     Traction   73 000  
    Marque   Modèle  year horsepower        road     boite     fuel cylindrée  \
0     Fiat  FIORINO  2020       5 CV      180 KM  Manuelle   Diesel         4 

In [785]:
# on va supprimer le feature "negiciable" de df_baniola car il n'a pas d'impact dans la prediction de prix des voiture.
df_baniola.drop('Negociable', axis=1, inplace=True)

In [786]:
# on va supprimer le feature "transmission" de df_automobile car il n'existe pas dans df_baniola
df_automobile.drop('transmission', axis=1, inplace=True)

In [787]:
print(df_baniola.head())
print('--------------------------------------------------------------------')
print(df_automobile.head())

    Marque   Modèle  year horsepower        road     boite     fuel cylindrée  \
0     Fiat  FIORINO  2020       5 CV      180 KM  Manuelle   Diesel         4   
1  Peugeot      308  2015       5 CV  180 000 KM  Manuelle  Essence         3   
2  Renault   Kangoo  2012       5 CV  240 000 KM  Manuelle   Diesel         6   
3   Suzuki   Autres  2020       5 CV   90 000 KM  Manuelle  Essence         3   
4   Suzuki   Autres  2020       5 CV   88 000 KM  Manuelle  Essence         3   

   nbre_portes   Prix  
0          3.0     30  
1          5.0  32000  
2          3.0  24500  
3          5.0  35800  
4          5.0  36500  
--------------------------------------------------------------------
          Marque             Modèle  year        road        boite horsepower  \
0  Mercedes-Benz           Classe E  2024        0 km  Automatique      10 cv   
1        Peugeot               2008  2014  162 000 km     Manuelle       5 cv   
2          Mazda                  6  2013  256 000 km  Au

In [788]:
# ajuster le prix :
df_automobile['Prix'] = df_automobile['Prix'].str.replace(' ', '')
print(df_automobile.head())


          Marque             Modèle  year        road        boite horsepower  \
0  Mercedes-Benz           Classe E  2024        0 km  Automatique      10 cv   
1        Peugeot               2008  2014  162 000 km     Manuelle       5 cv   
2          Mazda                  6  2013  256 000 km  Automatique       9 cv   
3           Ford  Ecosport Titanium  2022   60 000 km     Manuelle       7 cv   
4         Toyota         C-HR Sport  2020  162 000 km  Automatique       6 cv   

      fuel  nbre_portes  cylindrée    Prix  
0  Essence          4.0      1.400  430000  
1  Essence          4.0      2.398   36000  
2  Essence          5.0      0.999   33000  
3  Essence          5.0      1.497   68000  
4  Essence          4.0      1.999   73000  


In [789]:
# fusionner les 2 dataframes:
df = pd.concat([df_automobile, df_baniola], ignore_index=True)



In [790]:
print(df)

              Marque             Modèle  year        road        boite  \
0      Mercedes-Benz           Classe E  2024        0 km  Automatique   
1            Peugeot               2008  2014  162 000 km     Manuelle   
2              Mazda                  6  2013  256 000 km  Automatique   
3               Ford  Ecosport Titanium  2022   60 000 km     Manuelle   
4             Toyota         C-HR Sport  2020  162 000 km  Automatique   
...              ...                ...   ...         ...          ...   
17395           Opel             Kadett  1991       16 KM     Manuelle   
17396           Ford             FIESTA  2013  170 000 KM     Manuelle   
17397     Volkswagen               Golf  2010  276 000 KM     Manuelle   
17398      Chevrolet              CRUZE  2014  199 585 KM     Manuelle   
17399        Renault             Scenic  2007  299 000 KM     Manuelle   

      horsepower     fuel  nbre_portes cylindrée    Prix  
0          10 cv  Essence          4.0       1.4  43

In [791]:
# convertir le nbre de portes en int:
print(df['nbre_portes'].dtype)

# Remplacer NaN par 0 et convertir en int
df['nbre_portes'] = df['nbre_portes'].fillna(0).astype(int)
print(df)


float64
              Marque             Modèle  year        road        boite  \
0      Mercedes-Benz           Classe E  2024        0 km  Automatique   
1            Peugeot               2008  2014  162 000 km     Manuelle   
2              Mazda                  6  2013  256 000 km  Automatique   
3               Ford  Ecosport Titanium  2022   60 000 km     Manuelle   
4             Toyota         C-HR Sport  2020  162 000 km  Automatique   
...              ...                ...   ...         ...          ...   
17395           Opel             Kadett  1991       16 KM     Manuelle   
17396           Ford             FIESTA  2013  170 000 KM     Manuelle   
17397     Volkswagen               Golf  2010  276 000 KM     Manuelle   
17398      Chevrolet              CRUZE  2014  199 585 KM     Manuelle   
17399        Renault             Scenic  2007  299 000 KM     Manuelle   

      horsepower     fuel  nbre_portes cylindrée    Prix  
0          10 cv  Essence            4      

In [792]:
#  ajuster le kilometrage:*
print(df['road'].dtype)

# Remove ' km' and convert to float
df['road'] = df['road'].str.replace('km', '')
df['road'] = df['road'].str.replace('KM', '')
print(df)
print(df['road'].dtype)
print(df)



object
              Marque             Modèle  year      road        boite  \
0      Mercedes-Benz           Classe E  2024        0   Automatique   
1            Peugeot               2008  2014  162 000      Manuelle   
2              Mazda                  6  2013  256 000   Automatique   
3               Ford  Ecosport Titanium  2022   60 000      Manuelle   
4             Toyota         C-HR Sport  2020  162 000   Automatique   
...              ...                ...   ...       ...          ...   
17395           Opel             Kadett  1991       16      Manuelle   
17396           Ford             FIESTA  2013  170 000      Manuelle   
17397     Volkswagen               Golf  2010  276 000      Manuelle   
17398      Chevrolet              CRUZE  2014  199 585      Manuelle   
17399        Renault             Scenic  2007  299 000      Manuelle   

      horsepower     fuel  nbre_portes cylindrée    Prix  
0          10 cv  Essence            4       1.4  430000  
1         

In [793]:
# #  ajuster horsepower:*
print(df['horsepower'].dtype)
df['horsepower'] = df['horsepower'].str.replace('cv', '')
df['horsepower'] = df['horsepower'].str.replace('CV', '')
df['horsepower'] = df['horsepower'].str.replace(' ', '')

print(df)


object
              Marque             Modèle  year      road        boite  \
0      Mercedes-Benz           Classe E  2024        0   Automatique   
1            Peugeot               2008  2014  162 000      Manuelle   
2              Mazda                  6  2013  256 000   Automatique   
3               Ford  Ecosport Titanium  2022   60 000      Manuelle   
4             Toyota         C-HR Sport  2020  162 000   Automatique   
...              ...                ...   ...       ...          ...   
17395           Opel             Kadett  1991       16      Manuelle   
17396           Ford             FIESTA  2013  170 000      Manuelle   
17397     Volkswagen               Golf  2010  276 000      Manuelle   
17398      Chevrolet              CRUZE  2014  199 585      Manuelle   
17399        Renault             Scenic  2007  299 000      Manuelle   

      horsepower     fuel  nbre_portes cylindrée    Prix  
0             10  Essence            4       1.4  430000  
1         

In [794]:
# drop duplicates:
print(df.duplicated().sum())
df.drop_duplicates(inplace=True)
df.to_csv('final_data.csv', index=False)


15013


In [795]:
# renomer la colonne road en "kilometrage"
df.rename(columns={'road':'kilometrage'}, inplace=True)
print(df)

              Marque             Modèle  year kilometrage        boite  \
0      Mercedes-Benz           Classe E  2024          0   Automatique   
1            Peugeot               2008  2014    162 000      Manuelle   
2              Mazda                  6  2013    256 000   Automatique   
3               Ford  Ecosport Titanium  2022     60 000      Manuelle   
4             Toyota         C-HR Sport  2020    162 000   Automatique   
...              ...                ...   ...         ...          ...   
16195           Opel             Kadett  1991         16      Manuelle   
16196           Ford             FIESTA  2013    170 000      Manuelle   
16197     Volkswagen               Golf  2010    276 000      Manuelle   
16198      Chevrolet              CRUZE  2014    199 585      Manuelle   
16199        Renault             Scenic  2007    299 000      Manuelle   

      horsepower     fuel  nbre_portes cylindrée    Prix  
0             10  Essence            4       1.4  43

In [796]:
# unifier le feature cylindree : eliminer la lettre "L" et convertir à la meme unite 
df['kilometrage'] = df['kilometrage'].str.replace(' ', '')
print(df)


              Marque             Modèle  year kilometrage        boite  \
0      Mercedes-Benz           Classe E  2024           0  Automatique   
1            Peugeot               2008  2014      162000     Manuelle   
2              Mazda                  6  2013      256000  Automatique   
3               Ford  Ecosport Titanium  2022       60000     Manuelle   
4             Toyota         C-HR Sport  2020      162000  Automatique   
...              ...                ...   ...         ...          ...   
16195           Opel             Kadett  1991          16     Manuelle   
16196           Ford             FIESTA  2013      170000     Manuelle   
16197     Volkswagen               Golf  2010      276000     Manuelle   
16198      Chevrolet              CRUZE  2014      199585     Manuelle   
16199        Renault             Scenic  2007      299000     Manuelle   

      horsepower     fuel  nbre_portes cylindrée    Prix  
0             10  Essence            4       1.4  43

In [797]:
# save to csv :
df.to_csv('final_data.csv', index=False)
